# Creating a Sky Cube from an Input Scene

This notebook describes how to create a MIRISim scene within python, and then how to export that cube into a FITS file for further inspection.  This functionality is not available when MIRISim is used from the command line.

Any scene generated within MIRISim, and many scenes created using external files (FITS cubes or external text based SED files) can be exported to a FITS file, and here we show how to create a FITS cube for both imager and MRS outputs - which differ sigificantly in their fields of view, and number of required spectral channels.

Below are instructions to:

* create a sky scene using the MIRISim component SkySim
* output an MRS sky cube like that produced during a simulation
* output a more generic FITS file taken from the inputs, for both the imager and the MRS



## Setting up a Sky Scene

The first step will be to create a sky scene (including an extended galaxy, with a spectral line at 5.8 $\mu$m, a spectral energy distribution created by pySynPhot.

In [ ]:
#import mirisim components

from mirisim import skysim, obssim
#import the scene configuration parser
from mirisim.config_parser import SceneConfig
import numpy as np

In [ ]:
# create Background emission object
Background = skysim.Background(level='low')

#initialise the galaxy with a center at (-1,-1) arcsec,
# an axial ratio of 2, an effective radius of 0.5 arcsec, etc
Gal = skysim.Galaxy(Cen=(-1,-1),n=2,re=0.5,q=0.3,pa=40)

#setup line and continuum spectral energy distrubtions

# setup continuum spectral energy distribution from the 
# Bruzual & Charlot (1995) models in pysynphot
sedGALpysyn = skysim.wrap_pysynphot.PYSPSed(
                    family='bc95',sedname='bc95_b_10E6',wref=5.,flux=10.)

# attach the merged SED to the galaxy object
Gal.set_SED(sedGALpysyn)

scene = [Gal]



scene_config = SceneConfig.makeScene(loglevel=0,
                                    background =  Background,
                                    targets = scene)


# output the generated scene to a scene.ini file that could also be used as 
# input to a subsequent MIRISim simulation
scene_config.write('FITS_example_scene.ini')

## Creating a generic FITS file output

There is the more generalised example where the user would like to see the inputs to the simulation that have been generated by SkySim, regardless of whether they're using the Imager, LRS or MRS.  To do this, SkySim has a separate <code> makecube</code> function.  

For this function, the user needs to set the field of view (in arcsec), wavlength coverage range and both the spatial and spectral resolutions. To keep the output FITS file sizes reasonable, two different versions of the required code are presented below - one optimised for the Imager (with a larger field of view, and lower spectral resolution) and one for the MRS (with a smaller field of view and higher spectral resolution).

In [ ]:
##### Imager


FOV = np.array([[-57.,57.],[-57.,57.]])   # field of view [xmin,xmax],[ymin,ymax] (in arcsec)
SpatialSampling = 0.1               # spatial sampling (in arcsec)
WavelengthRange = [5,15]            # wavelength range to process (in microns)
WavelengthSampling = 0.5           # channel width (in microns)


scene = Gal + Background

# overwrite = True enables overwriting of any previous version of the fits file
# with the same name as that given in the writecube command
scene.writecube(cubefits = 'IMA_example_scene.fits',
               FOV = FOV, time = 0.0,
               spatsampling = SpatialSampling,
               wrange = WavelengthRange,
               wsampling = WavelengthSampling,
               overwrite = True)  



######## MRS

FOV = np.array([[-4.,4.],[-4.,4.]])   # field of view [xmin,xmax],[ymin,ymax] (in arcsec)
SpatialSampling = 0.1               # spatial sampling (in arcsec)
WavelengthRange = [5.2,6.1]            # wavelength range to process (in microns)
WavelengthSampling = 0.01           # channel width (in microns)


# overwrite = True enables overwriting of any previous version of the fits file
# with the same name as that given in the writecube command
scene.writecube(cubefits = 'MRS_example_scene.fits',
               FOV = FOV, time = 0.0,
               spatsampling = SpatialSampling,
               wrange = WavelengthRange,
               wsampling = WavelengthSampling,
               overwrite = True)  


The FITS files generated above could then subsequently be used as input to MIRISim as specified in a scene.ini file containing the following:

<code>
[sky]
    name       = FITScube               # Name of sky scene
    loglevel   = 1                      # 0: no log, 1: single summary, 2: full report

[Input FITS]
    Type = SkyCube
    cubefits = 'XXX_example_scene.fits'		# name of input fits file
</code>


## Create a FITS cube using the same methods used to produce the MRS skycubes

Now that the target scene has been created, we centre the MRS on its coordinates (here given to be  18h, 20deg). There are a number of steps to ensure the telescope is pointing at the right point on the sky:

* RA/DEC to centre on
* position angle of the observing field
* which optical path to follow (i.e. pointing through the centre of a specific MRS channel)
* translating those coordinates and optical path into the JWST internal coordinate system (v2,v3), and IFU positioning ($\alpha$,$\beta$)


the (v2,v3) coordinates are known for a specific MRS channel and sub-band configuration. In this case, because we're only creating a single pointing (e.g. not dithering), we will assume that any offsets in (v2,v3) in these observations are zero.

In [ ]:
ra_cen,dec_cen = '18:00:00',"20:00:00"
pa = 0

# MRS channel and sub-band to observe through
chan = 1
subband = 'SHORT'  # shortest wavelength observations

v2v3_ref = obssim.wcs.get_mrs_v2v3_ref(chan,subband)


# create pointing object from information above
Pointing = obssim.pointing.Pointing(ra_ref = ra_cen, dec_ref = dec_cen, pa = pa, 
                                    v2v3_off_commanded = (0,0), v2v3_off_actual = (0,0),
                                    v2v3_ref = v2v3_ref)

With an astronomical scene generated, and the telescope poiting defined, we next create a skycube for the specified channel and subband, and then export it to a FITS file. Note that for specifying the name of the FITS file, the channel and subband names are appended to the end of the given text.  Thus <code>galaxy</code> , in this case, becomes <code>galaxy1SHORT.fits</code>.

The <code> overwrite = True </code> option enables overwritting of an existing cube

In [ ]:
SkyCube = obssim.skycube.create_mrs_skycube_from_scene(scene = scene,pointing = Pointing,channel=chan,band=subband)
SkyCube.write_to_fits('galaxy',overwrite=True)